In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy


from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [2]:
# Load your existing database into a DataFrame
data = pd.read_csv("data/data_2018_08.2023.csv")  # Replace with the path to your database file
# Notice the .copy() to copy the values 
data = data.copy()

In [3]:
data.tail(3)

# To DO >>>
# de-emojify
# (done) drop lines with text = NaN
# concatinate text if same sender and similar time stamp
# stopwords

# create a dictionary with all BVG stations
# scrape https://www.bvg.de/de/verbindungen/stationsuebersicht ???


## Features:
# Line?
# Neighbourhood
# type of transport
# time of the day
# time of the month
# time of the year

# check another model where thy model the movement


# descriprive?
# predictive?

# bramian motion

# movement of fire

# movement of particles in the path?
# modelling it with the neural network
# diffusion models


# backup > heatmap of the findings
# dashboard / analysis of data
# probability model!



,group,sender,text,date
251277,freifahren_BE,5.608908e+08,Gehört eigentlich mindestens bos gemeldet...\n...,2023-08-18 12:41:15
251278,freifahren_BE,7.268913e+08,**3 boss **S Rathaus Spandau,2023-08-18 12:46:51
251279,freifahren_BE,5.535547e+09,"U7 Rudow, steigen gerade bei mehringdamm ein",2023-08-18 12:56:43


## Cleaning the dataset > Emojis, Dropping & Datetime

In [4]:
data["sender"].value_counts()

4.459087e+08    2241
3.708941e+08    1113
1.334822e+08     859
2.626739e+08     637
5.794010e+08     434
                ... 
1.027824e+09       1
9.530096e+08       1
6.359073e+08       1
5.806986e+07       1
6.577251e+09       1
Name: sender, Length: 28930, dtype: int64

In [5]:
data["group"].value_counts()

freifahren_BE    251280
Name: group, dtype: int64

In [6]:
len(data)

251280

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251280 entries, 0 to 251279
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   group   251280 non-null  object 
 1   sender  251279 non-null  float64
 2   text    206853 non-null  object 
 3   date    251280 non-null  object 
dtypes: float64(1), object(3)
memory usage: 7.7+ MB


In [8]:
data.describe() ### applicable only to the sender ID, does not make sense

,sender
count,2.512790e+05
mean,6.610125e+08
std,9.420384e+09
min,-1.001693e+12
25%,2.790832e+08
50%,5.541413e+08
75%,9.074052e+08
max,6.652919e+09


### Date > Datetime Format

In [9]:
data["date"] = pd.to_datetime(data["date"])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251280 entries, 0 to 251279
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   group   251280 non-null  object        
 1   sender  251279 non-null  float64       
 2   text    206853 non-null  object        
 3   date    251280 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 7.7+ MB


In [17]:
data[['text', 'date']].tail()

,text,date
251275,U5 Friedrichsfelde 2x blau Westen männlich gel...,2023-08-18 12:10:31
251276,U6 mehringdamm station.,2023-08-18 12:36:47
251277,Gehört eigentlich mindestens bos gemeldet...\n...,2023-08-18 12:41:15
251278,**3 boss **S Rathaus Spandau,2023-08-18 12:46:51
251279,"U7 Rudow, steigen gerade bei mehringdamm ein",2023-08-18 12:56:43


### De-Emojify

### Drop entries with no text

In [16]:
data = data[data['text'].notna()]

### Drop duplicates (there were 9 of them)

In [12]:
duplicate_count = data.duplicated().sum()
duplicate_count ### This is a lot of duplicates

9

In [13]:
data = data.drop_duplicates()
len(data)

206844

## Cleaning the dataset > Text Processing

In [14]:
data['text'] = data['text'].str.strip()
data['text']

1                                                        Hä
6                                                       YES
8                                                     😂😂  😂
11                         Da haben wir was ähnliches schon
13                                  So jetzt kanns losgehen
                                ...                        
251275    U5 Friedrichsfelde 2x blau Westen männlich gel...
251276                              U6 mehringdamm station.
251277    Gehört eigentlich mindestens bos gemeldet...\n...
251278                         **3 boss **S Rathaus Spandau
251279         U7 Rudow, steigen gerade bei mehringdamm ein
Name: text, Length: 206844, dtype: object

In [9]:

# List of words to search for
target_words = ['U7', 'U6']  # Add more stations as needed

# Iterate through each target word and filter the data
for word in target_words:
    keyword_rows = data[data['text'].str.contains(word)]
    filtered_data = keyword_rows.copy()

    # Save the filtered data to a new CSV file
    csv_filename = "filtered_data_{}.csv".format(word)
    filtered_data.to_csv(csv_filename, index=False)
    print("Filtered data for '{}' saved as: {}".format(word, csv_filename))

ValueError: Cannot mask with non-boolean array containing NA / NaN values